<a href="https://colab.research.google.com/github/PauDorMor/DPIEM/blob/main/PI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DPIEM
En el siguiente codigo se muestra y se trata de explicar el funcionamiento completo del codigo, el cual tiene la finalidad de poder clasificar e identificar EM disponibles en un espacio hospitalario

#Importe de librerias

In [ ]:
pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 40.7 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
from roboflow import Roboflow
import os
import re
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tqdm import tqdm
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.svm import LinearSVC
import pandas as pd
import joblib
from scipy.special import softmax


#Extracción de imagenes y carpetas de la base de datos

In [ ]:
#Datos de la sesion de roboflow
ROBOFLOW_API_KEY = "ysXFekfRklPYfXPY5kQa"
ROBOFLOW_WORKSPACE = "pau-dm-quidt"
ROBOFLOW_PROJECT = "medical-equipment-detection-t5ltj"
ROBOFLOW_VERSION = 3

rf = Roboflow(api_key="ysXFekfRklPYfXPY5kQa")
project = rf.workspace("pau-dm-quidt").project("medical-equipment-detection-t5ltj")

# Descarga del dataset, los datos estan en formato TF.Record
dataset = project.version(ROBOFLOW_VERSION).download("tfrecord")

print(f"Dataset downloaded to: {dataset.location}")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to medical-equipment-detection-3 in tfrecord:: 100%|██████████| 11/11 [00:04<00:00,  2.38it/s]


Dataset downloaded to: /content/medical-equipment-detection-3


In [ ]:
roboflow_dataset_path = dataset.location

print(f"Contenido de la Base de datos '{roboflow_dataset_path}':")
for root, dirs, files in os.walk(roboflow_dataset_path):
    level = root.replace(roboflow_dataset_path, '').count(os.sep)
    indent = ' ' * 4 * (level)
    print(f'{indent}{os.path.basename(root)}/')
    subindent = ' ' * 4 * (level + 1)
    for f in files:
        print(f'{subindent}{f}')

Contenido de la Base de datos '/content/medical-equipment-detection-3':
medical-equipment-detection-3/
    README.dataset.txt
    README.roboflow.txt
    valid/
        medical-euipment-detection.tfrecord
        medical-euipment-detection_label_map.pbtxt
    train/
        medical-euipment-detection.tfrecord
        medical-euipment-detection_label_map.pbtxt
    test/
        medical-euipment-detection.tfrecord
        medical-euipment-detection_label_map.pbtxt


In [ ]:
# Ruta de la base de datos en los archivos
Ruta_general = '/content/medical-equipment-detection-3'

# Ruta para la extracción de etiquetas, en este caso se utiliza la carpeta de entrenar, pero puede cambiar entre cualquiera
train_label_map_dir = os.path.join(Ruta_general, 'train')
label_map_filename = None

# Encuentro de datos en el dataset descargado (en formato pbtxt)
for file in os.listdir(train_label_map_dir):
    if file.endswith('_label_map.pbtxt'):
        label_map_filename = file
        break

label_map_path = os.path.join(train_label_map_dir, label_map_filename)

# Almacenamiento de las nombres de las clases tal y como vienen en roboflow
raw_class_names = []
label_id_to_raw_name = {}
with open(label_map_path, 'r') as f:
    content = f.read()

    items_blocks = re.findall(r'item\s*{([^}]*)}', content)
    for block in items_blocks:
        id_match = re.search(r'id:\s*(\d+)', block)
        name_match = re.search(r'display_name:\s*"([^"]*)"', block)
        if id_match and name_match:
            item_id = id_match.group(1)
            item_name = name_match.group(1)
            item_name_stripped = item_name.strip()
            label_id_to_raw_name[int(item_id)] = item_name_stripped
            raw_class_names.append(item_name_stripped)

# Enlistado de los nombres de las clases
unique_class_names = []
seen_names = set()
for name in raw_class_names:
    if name not in seen_names:
        unique_class_names.append(name)
        seen_names.add(name)

# Toma de referencia de los nombres que se usaran mas adelante
# Ademas de ello es para referenciarlo con el numero asociado a la clase que pertenece
class_names = unique_class_names
num_unique_classes = len(class_names)

#Asignacion entre el ID de la clase y el numero definido
label_id_to_unique_class_idx = {}
for label_id, raw_name in label_id_to_raw_name.items():
    if raw_name in class_names:
        label_id_to_unique_class_idx[label_id] = class_names.index(raw_name)
    else:
        # En caso de que no se encuentre una relacion entre el ID de los datos crudos y la imagen o dato, este no se eliminara, solo se indicara como desconocido
        label_id_to_unique_class_idx[label_id] = -1

print("Clases únicas detectadas :", class_names)
print("Número de clases:", num_unique_classes)
print("Numero de clase asociado a partir del ID de clase:", label_id_to_unique_class_idx)
print("Nombre y numero de clase :", {idx: name for idx, name in enumerate(class_names)})

# Variables globales uasadas para el downstream
num_classes = num_unique_classes

#Añade robuztes al modelo
keys_tensor = tf.constant(list(label_id_to_unique_class_idx.keys()), dtype=tf.int64)
values_tensor = tf.constant(list(label_id_to_unique_class_idx.values()), dtype=tf.int64)
initializer = tf.lookup.KeyValueTensorInitializer(keys_tensor, values_tensor)
label_lookup_table = tf.lookup.StaticHashTable(initializer, default_value=-1)


Clases únicas detectadas : ['Anesthesia Machine', 'BI', 'C-arm', 'Desf', 'ECG', 'ESU', 'EtO', 'MSV', 'US', 'VM', 'X-Ray']
Número de clases: 11
Numero de clase asociado a partir del ID de clase: {1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10}
Nombre y numero de clase : {0: 'Anesthesia Machine', 1: 'BI', 2: 'C-arm', 3: 'Desf', 4: 'ECG', 5: 'ESU', 6: 'EtO', 7: 'MSV', 8: 'US', 9: 'VM', 10: 'X-Ray'}


Todo el siguiente codigo seria como un post-edicion para correlacionar las clases, esto servira para la extraccion de caracteristicas y la correlacion entre la imagen y los datos que se encuentren en ella, ademas de la clase a la que pertenece

In [ ]:
# Define la carpeta de la cual se extraeran las imagenes para la evaluacion de modelo
tfrecord_file_path = os.path.join(Ruta_general, 'train', 'medical-euipment-detection.tfrecord')

def parse_tf_example(example_proto):
    # Sienta las bases para la extraccion de caracteristicas
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
        'image/height': tf.io.FixedLenFeature([], tf.int64),
        'image/width': tf.io.FixedLenFeature([], tf.int64),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
    }

    example = tf.io.parse_single_example(example_proto, feature_description)

    #Renombra a img_tensor para evitar sobreponer los datos
    img_tensor = tf.image.decode_jpeg(example['image/encoded'], channels=3)
    img_tensor = tf.image.resize(img_tensor, [224, 224])
    img_tensor = tf.cast(img_tensor, tf.float32)

    labels_tensor = example['image/object/class/label'].values

    #Comprueba que hayan todas las etiquetas, en caso de que no esten, las evalua mas tarde y se indican con -1
    if tf.size(labels_tensor) > 0:
        raw_label_id = labels_tensor[0]
        unique_class_idx = label_lookup_table.lookup(raw_label_id)
    else:
        unique_class_idx = tf.constant(-1, dtype=tf.int64)

    return img_tensor, unique_class_idx

# Creacion del dataset
raw_dataset = tf.data.TFRecordDataset(tfrecord_file_path)

# Mapea el emparejamiento de los dos datos
parsed_dataset_with_sentinel = raw_dataset.map(parse_tf_example)

# Filtra los datos sin clase, asigna -1
parsed_dataset = parsed_dataset_with_sentinel.filter(lambda image, label: tf.not_equal(label, -1))

#Muestra algunos ejemplos
print("\n Algunos ejemplos del dataset:")
class_idx_to_name_map_for_print = {idx: name for idx, name in enumerate(class_names)}
for image, unique_class_idx in parsed_dataset.take(3):
    print(f"  Datos de la imagen: {image.shape}, Numero de clase: {unique_class_idx.numpy()} (Class: {class_idx_to_name_map_for_print.get(unique_class_idx.numpy(), 'Unknown')})")



 Algunos ejemplos del dataset:
  Datos de la imagen: (224, 224, 3), Numero de clase: 8 (Class: US)
  Datos de la imagen: (224, 224, 3), Numero de clase: 9 (Class: VM)
  Datos de la imagen: (224, 224, 3), Numero de clase: 3 (Class: Desf)


#Inicio de modelo de extracción de caracteristicas

In [ ]:
#Inicio del modelo de extraccion de caracteristicas
#top=false ayuda a eliminar capas que no aporten al modelo
#pooling=avg ayuda a la formacion del vector de caracteristicas
model_vgg = VGG16(weights='imagenet', include_top=False, pooling='avg')



58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
#tamaño del lote
BATCH_SIZE = 32

# Inicio de procesamiento por VGG16
def apply_vgg16_preprocessing_and_label_mapping(image, unique_class_idx):
    image = vgg16_preprocess(image)
    return image, unique_class_idx
#Se aplica el procesamiento aleatorio por lotes y los datos unicos de cada imagen
train_dataset = parsed_dataset.map(apply_vgg16_preprocessing_and_label_mapping)

#Mezcla, agrupa y precarga las imagenes

train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"Dataset cuenta con {BATCH_SIZE} imagenes por lote")
print(f"Numero de clases unicas: {num_classes}")




Dataset cuenta con 32 imagenes por lote
Numero de clases unicas: 11


In [ ]:
# Inicializa listas para las caracteristicas y las etiquetas
all_features = []
all_labels = []

print("Extracting features from the dataset using VGG16...")

#Itera el conjunto de datos preprocesados para extraer sus caracteristicas
#Utiliza VGG16 sin capas superiores para la extraccion de caracteristicas
for images, labels in tqdm(train_dataset):
    features = model_vgg.predict(images, verbose=0)

   #Amplia la lista con las caracteristicas y etiquetas asociadas
    all_features.extend(features)
    all_labels.extend(labels.numpy())

#Convierte la lista a datos de numpy
all_features = np.array(all_features)
all_labels = np.array(all_labels)

print(f"Extraccion de caracteristicas completa, son : {len(all_labels)}")
print(f"Forma de las caracteristicas y las etiquetas: {all_features.shape}")



Extracting features from the dataset using VGG16...


22it [07:09, 19.51s/it]

Extraccion de caracteristicas completa, son : 684
Forma de las caracteristicas y las etiquetas: (684, 512)


Entrenamiento y evaluacion del modelo de clasificación

In [ ]:

#Inicializa el nummero de elementos a extraer
accuracies = []
precisions = []
recalls = []
f1s = []
kappas = []

#Define el clasificador y se utiliza el modelo SVC
clf = LinearSVC(random_state=42, dual=True, max_iter=1000000)

#Define el numero de "ciclos" de evaluacion del modelo
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_index, test_index) in enumerate(skf.split(all_features, all_labels)):
    print(f"\n--- Ciclo {fold+1} ---")
    X_train, X_test = all_features[train_index], all_features[test_index]
    y_train, y_test = all_labels[train_index], all_labels[test_index]

    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)

    accuracy = metrics.accuracy_score(y_test, y_pred)
    precision = metrics.precision_score(y_test, y_pred, average='macro', zero_division=0)
    recall = metrics.recall_score(y_test, y_pred, average='macro', zero_division=0)
    f1 = metrics.f1_score(y_test, y_pred, average='macro', zero_division=0)
    kappa = metrics.cohen_kappa_score(y_test, y_pred)

    accuracies.append(accuracy)
    precisions.append(precision)
    recalls.append(recall)
    f1s.append(f1)
    kappas.append(kappa)

    print(f"Precisión: {accuracy:.4f}")
    print(f"Exactitud (macro): {precision:.4f}")
    print(f"Recall (macro): {recall:.4f}")
    print(f"F1-Score (macro): {f1:.4f}")
    print(f"Kappa: {kappa:.4f}")

print("\n--- Cross-Validation Results ---")
print(f"Precision promedio: {np.mean(accuracies):.4f} (Std: {np.std(accuracies):.4f})")
print(f"Exactitud promedio: {np.mean(precisions):.4f} (Std: {np.std(precisions):.4f})")
print(f"Recall promedio: {np.mean(recalls):.4f} (Std: {np.std(recalls):.4f})")
print(f"F1 promedio: {np.mean(f1s):.4f} (Std: {np.std(f1s):.4f})")
print(f"Kappa promedio: {np.mean(kappas):.4f} (Std: {np.std(kappas):.4f})")

#Re entrenamiento de todo el dataset
final_clf = LinearSVC(random_state=42, dual=True, max_iter=1000000)
final_clf.fit(all_features, all_labels)
print("Clasificador entrenado con todo el dataset.")



--- Ciclo 1 ---
Precisión: 0.8613
Exactitud (macro): 0.8832
Recall (macro): 0.8667
F1-Score (macro): 0.8685
Kappa: 0.8437

--- Ciclo 2 ---
Precisión: 0.7664
Exactitud (macro): 0.7794
Recall (macro): 0.7312
F1-Score (macro): 0.7436
Kappa: 0.7367

--- Ciclo 3 ---
Precisión: 0.7883
Exactitud (macro): 0.7346
Recall (macro): 0.7386
F1-Score (macro): 0.7327
Kappa: 0.7619

--- Ciclo 4 ---
Precisión: 0.7737
Exactitud (macro): 0.7605
Recall (macro): 0.7795
F1-Score (macro): 0.7522
Kappa: 0.7466

--- Ciclo 5 ---
Precisión: 0.7206
Exactitud (macro): 0.7178
Recall (macro): 0.6819
F1-Score (macro): 0.6918
Kappa: 0.6855

--- Cross-Validation Results ---
Precision promedio: 0.7821 (Std: 0.0456)
Exactitud promedio: 0.7751 (Std: 0.0580)
Recall promedio: 0.7596 (Std: 0.0619)
F1 promedio: 0.7577 (Std: 0.0591)
Kappa promedio: 0.7549 (Std: 0.0513)
Clasificador entrenado con todo el dataset.


In [ ]:
# Ruta para guardar el clasificador
classifier_save_path = '/content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib'

# Obtener el directorio de la ruta del archivo
save_directory = os.path.dirname(classifier_save_path)

# Crear el directorio si no existe
os.makedirs(save_directory, exist_ok=True)

# Guardar el clasificador
joblib.dump(final_clf, classifier_save_path)
print(f"Clasificador guardado en : {classifier_save_path}")

Clasificador guardado en : /content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib


#Puesta en marcha del modelo

Se carga el modelo preentrenado para la extraccion de caracteristicas, ademas que hace una doble verificacion del modelo de entrenamiento

In [ ]:
model_vgg_inference = VGG16(weights='imagenet', include_top=False, pooling='avg')

classifier_load_path = '/content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib'
loaded_clf = joblib.load(classifier_load_path)

#Nueva ruta de imagen a clasificar
new_image_path = '/content/Radiopaco.png'

# Preprocesamiento de la imagen
def extract_features_single_image(img_path, feature_extractor_model, preprocess_func):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_func(x)
    features = feature_extractor_model.predict(x, verbose=0)
    return features.flatten()

# Ectraccion de caracteristicas de la nueva imagen
print(f"Extraccion de caracteristicas de la imagen : {new_image_path}")
new_image_features = extract_features_single_image(new_image_path, model_vgg_inference, vgg16_preprocess)

# Reestructuracion de la imagen para hacer la prediccion
new_image_features = new_image_features.reshape(1, -1)

# Inicio de la prediccion
predicted_label_id = loaded_clf.predict(new_image_features)[0]

#Relacion entre el nombre de la clase utilizando nombres globales
# En caso de que no encuentre el nombre de la clase, lo menciona
if 0 <= predicted_label_id < len(class_names):
    predicted_class_name = class_names[predicted_label_id]
    print(f"ID de clase predecida: {predicted_label_id}, Clase predecida: {predicted_class_name}")
else:
    print(f"ID de clase predecida: {predicted_label_id}, No hay nombre de clase asociado.")

print("Prediccion completa.")

Extraccion de caracteristicas de la imagen : /content/Radiopaco.png
ID de clase predecida: 7, Clase predecida: MSV
Prediccion completa.


Modelo modificado con %

In [ ]:
model_vgg_inference = VGG16(weights='imagenet', include_top=False, pooling='avg')

classifier_load_path = '/content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib'
loaded_clf = joblib.load(classifier_load_path)
print(f"Clasificador cargado de : {classifier_load_path}")

#Incluye aqui la ruta de la imagen a evaluar
new_image_path = '/content/C-Arm.webp'

def extract_features_single_image(img_path, feature_extractor_model, preprocess_func):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=(224, 224))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0) # agrega la dimensión del conjunto de imágenes
    x = preprocess_func(x)
    features = feature_extractor_model.predict(x, verbose=0)
    return features.flatten()

print(f"Extrayendo caracteristicas de la nueva imagen: {new_image_path}")
new_image_features = extract_features_single_image(new_image_path, model_vgg_inference, vgg16_preprocess)

new_image_features = new_image_features.reshape(1, -1)

predicted_label_id = loaded_clf.predict(new_image_features)[0]

# Da un valor dependiendo al nivel de emparejamiento resultante
decision_scores = loaded_clf.decision_function(new_image_features)[0] # obtiene la puntuación de la muestra simple

#Se da la confianza del modelo como porcentaje, mas no es el valor de fidelidad del modelo
confidences = softmax(decision_scores)

# Obtencion de la confianza del modelo y lo convierte a porcentaje
predicted_confidence = confidences[predicted_label_id] * 100

if 0 <= predicted_label_id < len(class_names):
    predicted_class_name = class_names[predicted_label_id]
    print(f"ID de clase predicho: {predicted_label_id}, Clase predicha: {predicted_class_name}")
    print(f"Confianza de modelo (aprox): {predicted_confidence:.2f}%")
else:
    print(f"ID de clase: {predicted_label_id}, No se encontro una clase con esos datos.")

print("Prediccion completa.")


Clasificador cargado de : /content/drive/MyDrive/A - FPDI/medical_equipment_classifier.joblib
Extrayendo caracteristicas de la nueva imagen: /content/C-Arm.webp
ID de clase predicho: 2, Clase predicha: C-arm
Confianza de modelo (aprox): 54.48%
Prediccion completa.


# *El codigo fue realizado con ayuda de Gemini Ai*